In [1]:
__author__ = "Pattarapon 4012, Suchada 4016, Pawornrat 4030"

## Import Models

In [2]:
import gurobipy
import numpy as np
import pandas as pd
import altair as alt
import datapane as dp
from typing import List, Dict

## Gurobi Optimizations

In [3]:
def optimize_planning(timeline: List[str],
                      rigs: List[str], 
                      yearly_requirements: Dict[str, int], 
                      production_per_rig: Dict[str, int],
                      API_cap: Dict[str, int]) -> pd.DataFrame:
    
    model = gurobipy.Model("Optimize production planning")
    
 ###############################################################

    # DEFINE VARIABLES   
    line_opening = model.addVars(
        timeline, rigs, vtype=gurobipy.GRB.BINARY, name="Use status"
    )

    oil_produced = model.addVars(
        timeline,
        rigs,
        vtype=gurobipy.GRB.CONTINUOUS,
        name="Crude Oil Production",
    )
    
    sum_of_oil_produced = model.addVars(
        timeline,
        vtype=gurobipy.GRB.CONTINUOUS,
        name="Crude Oil Divide",
    )
    
    API_Mul_Oil = model.addVars(
        timeline,
        rigs,
        vtype=gurobipy.GRB.CONTINUOUS,
        name="API Mul Oil Produced",
    )
    
    
###############################################################

    # CONSTRAINTS
    model.addConstrs(
        (
            oil_produced[(date, rig)] <= production_per_rig[rig] * line_opening[(date, rig)]
            for date in timeline
            for rig in rigs
        ),
        name="Oil produced - Barrels per year",
    )

    model.addConstrs(
        (
            line_opening.sum(date, "*") <= 3
            for date in timeline
        ),
        name="Restricted to 3 rigs per year",
    )

    model.addConstrs(
        (
            oil_produced.sum(date, '*') == sum_of_oil_produced[(date)]
            for date in timeline
        ),
        name="Divide of oil produced",
    )

    model.addConstrs(
        (
            API_Mul_Oil[(date, rig)] == API_cap[rig]*oil_produced[(date, rig)]
            for date in timeline
            for rig in rigs
        ),
        name="API Mul Oil",
    )

    model.addConstrs(
        (
            gurobipy.quicksum(API_Mul_Oil[(date, rig)] for rig in rigs)
            == yearly_requirements[date] * sum_of_oil_produced[(date)]
            for date in timeline
            
        ),
        name="Oil produced API - Requirement",
    )
    
###############################################################

    # DEFINE MODEL
    # Objective : maximize a function
    model.ModelSense = gurobipy.GRB.MAXIMIZE
 
    # Function to maximize [maximize the sum]
    optimization_var = gurobipy.quicksum(
        oil_produced[(date, rig)] for date in timeline for rig in rigs
    )
    objective = 0
    objective += optimization_var
    
###############################################################
    
    # SOLVE MODEL
    model.setObjective(objective)
    model.optimize()
    sol = pd.DataFrame(data={"Solution": model.X}, index=model.VarName)
    
    print("Total produced = " + str(model.ObjVal) + " barrels")
        
    return sol, model

## Altair and Datapane graph plotting

In [4]:
def plot_planning(planning, timeline):
    
    # Plot graph - Oil each rig produced
    source = planning.filter(like="Crude Oil Production",axis=0).copy()
    source["Date"] = list(source.index.values)
    source = source.rename(columns={"Solution": "Barrels"}).reset_index()
    source[["Date", "Rig"]] = source["Date"].str.split(",", expand=True)
    source["Date"] = source["Date"].str.split("[").str[1]
    source["Rig"] = source["Rig"].str.split("]").str[0]
    bars = (
        alt.Chart(source)
        .mark_bar()
        .encode(
            x="Rig:N",
            y="Barrels:Q",
            column=alt.Column("Date:N"),
            color="Rig:N",
            tooltip=["Date", "Rig", "Barrels"],
        )
        .interactive()
        .properties(
            width=550 / len(timeline) - 22,
            height=150,
            title="Optimized Production of each rig"
        )
    )
    
    # Plot graph - Total oil produced
    source2 = source.groupby("Date")["Barrels"].sum().reset_index()
    bars_sum = (
        alt.Chart(source2)
        .mark_bar()
        .encode(
            y=alt.Y("Barrels", axis=alt.Axis(grid=False)),
            column=alt.Column("Date:N"),
            color="Date:N",
            tooltip=["Date", "Barrels"],
        )
        .interactive()
        .properties(
            width=550 / len(timeline) - 22,
            height=75,
            title="Total Production"
        )
    )
    
    # Plot graph - Raw revenue
    source3 = source.groupby("Date")["Barrels"].sum().reset_index()
    source3["Raw Rev"] = source3["Barrels"]*75
        
    bars_rawrev = (
        alt.Chart(source3)
        .mark_bar()
        .encode(
            y=alt.Y("Raw Rev", axis=alt.Axis(grid=False)),
            column=alt.Column("Date:N"),
            color="Date:N",
            tooltip=["Date", "Raw Rev"],
        )
        .interactive()
        .properties(
            width=550 / len(timeline) - 22,
            height=75,
            title="Raw Revenue"
        )
    )
    
    # Plot graph - Adjusted revenue
    # raw_rev*(1-0.05)^year-1 - royalties*(1+0.02)^year-1 
    source4 = planning.filter(like="Use status",axis=0).copy()
    source4["Date"] = list(source4.index.values)
    source4 = source4.rename(columns={"Solution": "Usage"}).reset_index()
    source4[["Date", "Rig"]] = source4["Date"].str.split(",", expand=True)
    source4["Date"] = source4["Date"].str.split("[").str[1]
    source4["Rig"] = source4["Rig"].str.split("]").str[0]
    source4["Year"] = source4["Date"].str.split("/").str[0]
    source4["Year"] = pd.to_numeric(source4["Year"])
    source4["Royalties"] = source4["Year"] # Dummy
    
    # Calculate royalities by rig and year
    for index, row in source4.iterrows():
        if(source4["Rig"][index] == "RigA"):
            source4["Royalties"][index] = source4["Usage"][index]*(13085600)*((1+0.02)**(source4["Year"][index]-2015))
        elif(source4["Rig"][index] == "RigB"):
            source4["Royalties"][index] = source4["Usage"][index]*(16357000)*((1+0.02)**(source4["Year"][index]-2015))
        elif(source4["Rig"][index] == "RigC"):
            source4["Royalties"][index] = source4["Usage"][index]*(13085600)*((1+0.02)**(source4["Year"][index]-2015))
        elif(source4["Rig"][index] == "RigD"):
            source4["Royalties"][index] = source4["Usage"][index]*(16357000)*((1+0.02)**(source4["Year"][index]-2015))
            
    source4 = source4.groupby("Date")["Royalties"].sum().reset_index()
    source4["Raw Rev"] = source3["Raw Rev"]
    source4["Pre adj Rev"] = source4["Raw Rev"] # Dummy
    
    for index, row in source4.iterrows():
        source4["Pre adj Rev"][index] *= (1-0.05)**index

    source4["Adjusted Rev"] = source4["Pre adj Rev"]-source4["Royalties"]
    print(source4)

    
    bars_adjustrev = (
        alt.Chart(source4)
        .mark_bar()
        .encode(
            y=alt.Y("Adjusted Rev", axis=alt.Axis(grid=False)),
            column=alt.Column("Date:N"),
            color="Date:N",
            tooltip=["Date", "Adjusted Rev"],
        )
        .interactive()
        .properties(
            width=550 / len(timeline) - 22,
            height=75,
            title="Adjusted Revenue"
        )
    )
    
    
    # Vertically Concatenate all graphs
    chart = alt.vconcat(bars, bars_sum, bars_rawrev, bars_adjustrev)
    chart.save("Oil_plan_model_4.html")

    dp.Report(dp.Plot(chart, caption="Oil production model 4")).upload(
        name="Oil production model 4",
        description="Oil production model 4 by Samkok",
        open=True,
        visibily="PUBLIC",
    )

## Executing requirements and informations

In [5]:
# To get maximum oil production each year, comment other years away

# Year 1 (2015)
yearly_requirements: Dict[str, int] = {
    "2015/01/01": 27,
    "2016/01/01": 24,
    "2017/01/01": 36,
    "2018/01/01": 18,
    "2019/01/01": 30
}

yearly_requirements_df = pd.DataFrame.from_dict(yearly_requirements, orient="index")

calendar: List[str] = list(yearly_requirements.keys())

production_per_rig = {"RigA": 1900000, "RigB": 1500000, "RigC": 1000000, "RigD": 2300000}

rigs: List[str] = list(production_per_rig.keys())

API_cap = {"RigA": 21, "RigB": 30, "RigC": 45, "RigD": 15}

solution, model = optimize_planning(
    calendar,
    rigs,
    yearly_requirements,
    production_per_rig,
    API_cap
)
# print(solution)
# solution.to_csv('Optimized_test.csv', encoding='utf-8')
plot_planning(solution, calendar)
# print(model.ObjVal)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-07
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 55 rows, 65 columns and 150 nonzeros
Model fingerprint: 0x5911509d
Variable types: 45 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 49 rows and 57 columns
Presolve time: 0.01s
Presolved: 6 rows, 8 columns, 16 nonzeros
Found heuristic solution: objective 1.556667e+07
Variable types: 4 continuous, 4 integer (4 binary)

Root relaxation: objective 2.020648e+07, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

C:\Users\computer\AppData\Local\Temp\ipykernel_14348\3965158374.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source4["Royalties"][index] = source4["Usage"][index]*(13085600)*((1+0.02)**(source4["Year"][index]-2015))
C:\Users\computer\AppData\Local\Temp\ipykernel_14348\3965158374.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source4["Royalties"][index] = source4["Usage"][index]*(16357000)*((1+0.02)**(source4["Year"][index]-2015))
C:\Users\computer\AppData\Local\Temp\ipykernel_14348\3965158374.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats

Uploading report and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a single report? More info <a href='https://docs.datapane.com/reports/blocks/layout-pages-and-selects' target='_blank'>here</a>

Report successfully uploaded. View and share your report <a href='https://datapane.com/u/heartnoxill/reports/mA2EMak/oil-production-model-4/' target='_blank'>here</a>, or edit your report <a href='https://datapane.com/u/heartnoxill/reports/mA2EMak/oil-production-model-4/edit/' target='_blank'>here</a>.